### Setup

In [1]:
# this first setup block will pretty much always stay the same
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_create_multiple_reports, do_fit_scenarios
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20220830_scenario_projections.ipynb")

2022/09/20 22:13:54 INFO|============================================================


### Fit an initial scenario through Feb 2022

In [3]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20220815_aug_gov_briefing/20220815_scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/input/vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2024-01-01',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-02-28' #'2022-02-28'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

2022/09/20 22:13:58 INFO|-------------------------|{} Retrieving vaccinations data
2022/09/20 22:14:01 INFO|-------------------------|{} Constructing vaccination projections
2022/09/20 22:14:04 INFO|-------------------------|{} Retrieving hospitalizations data
2022/09/20 22:14:07 INFO|----------------------|{} Prepping Model
2022/09/20 22:14:07 INFO|-----------------------|{} Prepping Model
2022/09/20 22:15:18 INFO|----------------------|{} Model prepped for fitting in 71.01421341599999 seconds.
2022/09/20 22:15:18 INFO|----------------------|{} Will fit 25 times
2022/09/20 22:16:31 INFO|----------------------|{'fit_batch': '0'}: Transmission control fit 1/25 completed in 73.881068833 seconds: {0: {'co': 0.8587936465412482}, 14: {'co': 0.6897974169375691}, 28: {'co': 0.5520416889747688}, 42: {'co': 0.0012499217244474978}, 56: {'co': 0.6815644007422084}, 70: {'co': 0.8912220172459057}}
2022/09/20 22:16:31 INFO|-----------------------|{'fit_batch': '0'}: Running forward sim
2022/09/20 22

2022/09/20 22:21:41 INFO|----------------------|{'fit_batch': '18'}: Transmission control fit 19/25 completed in 9.847199875000001 seconds: {504: {'co': 0.7862064714343785}, 518: {'co': 0.7799209533639639}, 532: {'co': 0.7570481774472059}, 546: {'co': 0.7091713379237341}, 560: {'co': 0.7430708101284107}, 574: {'co': 0.7376115689086383}}
2022/09/20 22:21:41 INFO|-----------------------|{'fit_batch': '18'}: Running forward sim
2022/09/20 22:21:57 INFO|----------------------|{'fit_batch': '19'}: Transmission control fit 20/25 completed in 13.998242957999992 seconds: {532: {'co': 0.7562230410522897}, 546: {'co': 0.7112566962456192}, 560: {'co': 0.7393433618720686}, 574: {'co': 0.7492674122336548}, 588: {'co': 0.7862561758011867}, 602: {'co': 0.7652667074352255}}
2022/09/20 22:21:57 INFO|-----------------------|{'fit_batch': '19'}: Running forward sim
2022/09/20 22:22:09 INFO|----------------------|{'fit_batch': '20'}: Transmission control fit 21/25 completed in 10.635648791999984 seconds: 

### Create and run scenarios from Feb 2022 to present

In [4]:
multiprocess = 4

scenario_params = json.load(open("covid_model/analysis/20220815_aug_gov_briefing/20220815_scenario_params.json"))

model_args = {
    'base_spec_id': 3995 #model.spec_id # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# compute vacc effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
#for hosp_frac in [0.7, 0.8, 0.9]:
    #for school_contact_increase in [1.0, 1.1, 1.2]:
        #for beta_mult in [0.95, 1.0, 1.05]:
for vx_seed in [0, 2]:
    for vir_mult in [0.833, 2.38]:
        hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*0.8), "2022-03-15": (0.34 + 0.66*0.8), "2022-03-30": 0.8}
            #beta_param_adjustment = [{"param": "betta",
                                      #"attrs": {"variant": "ba45"},
                                      #"mults": {"2020-01-01": beta_mult},
                                      #"desc": "sensitivity analysis for beta"}]
            #school_contact = [{"param": "betta",
                               #"attrs": {'age': '0-19'},
                               #"mults": {"2020-01-01": 1, "2022-08-15": school_contact_increase},
                               #"desc": "After August 15th, beta increases by a factor of the school contact increase"}]
        vx_adjust = [{"param": "vx_seed",
                      "vals": {"2020-01-01": 0, "2022-09-20": vx_seed, "2022-10-20": 0},
                      "desc": "Variant X seeding"}]
        vir_adjust = [{"param": "hosp",
                       "attrs": {"variant": "vx"},
                       "mults": {"2020-01-01": vir_mult},
                       "desc": "Variant X hospitalization multiplier"}]
        lt5_vacc_adjust = [{"param": "immunity",
                                "attrs": {'age': '0-19', 'vacc': 'shot1'},
                                "mults": {"2020-01-01": 1, "2022-06-20": 0.99 + 0.01*vacc_eff_lt5,
                                          "2022-06-30": 0.98 + 0.02*vacc_eff_lt5,
                                          "2022-07-10": 0.97 + 0.03*vacc_eff_lt5,
                                          "2022-07-20": 0.96 + 0.04*vacc_eff_lt5,
                                          "2022-07-30": 0.955 + 0.045*vacc_eff_lt5,
                                          "2022-08-10": 0.95 + 0.05*vacc_eff_lt5,
                                          "2022-08-20": 0.945 + 0.055*vacc_eff_lt5,
                                          "2022-08-30": 0.94 + 0.06*vacc_eff_lt5,
                                          "2022-09-10": 0.935 + 0.065*vacc_eff_lt5,
                                          "2022-09-20": 0.93 + 0.07*vacc_eff_lt5},
                                "desc": "weighted average using share of 0-19 getting shot1 who are under 5"}]
        scenario_model_args.append({'params_defs': scenario_params + vir_adjust + vx_adjust+ lt5_vacc_adjust,
                                    'hosp_reporting_frac': hrf,
                                    'tags': {'vx_seed': vx_seed,
                                             'vir_mult': vir_mult}})
            
            

In [5]:
# check how many scenarios there are
len(scenario_model_args)

4

In [6]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

2022/09/20 22:25:08 INFO|============================================================
2022/09/20 22:25:08 INFO|============================================================
2022/09/20 22:25:08 INFO|============================================================
2022/09/20 22:25:08 INFO|============================================================
2022/09/20 22:25:12 INFO|-----------|{'vx_seed': 2, 'vir_mult': 2.38} Retrieving vaccinations data
2022/09/20 22:25:12 INFO|-----------|{'vx_seed': 2, 'vir_mult': 0.833} Retrieving vaccinations data
2022/09/20 22:25:12 INFO|-----------|{'vx_seed': 0, 'vir_mult': 2.38} Retrieving vaccinations data
2022/09/20 22:25:12 INFO|-----------|{'vx_seed': 0, 'vir_mult': 0.833} Retrieving vaccinations data
2022/09/20 22:25:15 INFO|-----------|{'vx_seed': 2, 'vir_mult': 0.833} Constructing vaccination projections
2022/09/20 22:25:15 INFO|-----------|{'vx_seed': 0, 'vir_mult': 2.38} Constructing vaccination projections
2022/09/20 22:25:15 INFO|-----------|{'vx_s

2022/09/20 22:27:39 INFO|--------|{'vx_seed': 0, 'vir_mult': 0.833, 'fit_batch': '2'}: Transmission control fit 3/5 completed in 13.987294082999995 seconds: {823: {'co': 0.8484813092112764}, 837: {'co': 0.8463817396766858}, 851: {'co': 0.8329165472400655}, 865: {'co': 0.8618380033604983}, 879: {'co': 0.8535020162621955}, 893: {'co': 0.8576951786786402}}
2022/09/20 22:27:39 INFO|---------|{'vx_seed': 0, 'vir_mult': 0.833, 'fit_batch': '2'}: Running forward sim
2022/09/20 22:27:40 INFO|--------|{'vx_seed': 0, 'vir_mult': 2.38, 'fit_batch': '2'}: Transmission control fit 3/5 completed in 14.39388220799998 seconds: {823: {'co': 0.8484813092112764}, 837: {'co': 0.8463817396766858}, 851: {'co': 0.8329165472400655}, 865: {'co': 0.8618380033604983}, 879: {'co': 0.8535020162621955}, 893: {'co': 0.8576951786786402}}
2022/09/20 22:27:40 INFO|---------|{'vx_seed': 0, 'vir_mult': 2.38, 'fit_batch': '2'}: Running forward sim
2022/09/20 22:27:56 INFO|--------|{'vx_seed': 2, 'vir_mult': 2.38, 'fit_bat

2022/09/20 22:28:20 INFO|--------|{'vx_seed': 0, 'vir_mult': 2.38, 'fit_batch': '4', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.858793645564471}, 14: {'co': 0.6897974238901571}, 28: {'co': 0.5461562373730889}, 42: {'co': 1.488463975320554e-14}, 56: {'co': 0.6964648244185871}, 70: {'co': 0.8333063816292616}, 84: {'co': 0.8295949264098259}, 98: {'co': 0.8317180855515073}, 112: {'co': 0.8774454485555294}, 126: {'co': 0.8243715707940461}, 140: {'co': 0.7604768918170302}, 154: {'co': 0.6712541505311671}, 168: {'co': 0.8430274763680383}, 182: {'co': 0.7826504009954449}, 196: {'co': 0.8191652900355885}, 210: {'co': 0.7726023569388112}, 224: {'co': 0.7713430227931438}, 238: {'co': 0.7185520655720183}, 252: {'co': 0.72213950524156}, 266: {'co': 0.6850199052463005}, 280: {'co': 0.7198410425845795}, 294: {'co': 0.7578492868967014}, 308: {'co': 0.8225928703403277}, 322: {'co': 0.8115175392447233}, 336: {'co': 0.7825644083248113}, 350: {'co': 0.795399515500132}, 364: {'co': 0.8054754975658831}, 37

### Run the report for each fit model

In [7]:
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir, prep_model=False, solve_model=True, immun_variants=['ba45'], from_date='2022-01-01')

2022/09/20 22:29:04 INFO|============================================================
2022/09/20 22:29:04 INFO|--------|Solving model
2022/09/20 22:29:05 INFO|============================================================
2022/09/20 22:29:05 INFO|--------|Solving model
2022/09/20 22:29:05 INFO|============================================================
2022/09/20 22:29:05 INFO|--------|Solving model
2022/09/20 22:29:06 INFO|============================================================
2022/09/20 22:29:06 INFO|--------|Solving model
2022/09/20 22:29:09 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/09/20 22:29:09 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/09/20 22:29:10 WARNING|No artists with labels found to put in legend.  Note that artists 

In [8]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

2022/09/20 22:31:47 INFO|Projecting
2022/09/20 22:31:47 INFO|
2022/09/20 22:32:05 INFO|{'vx_seed': 0, 'vir_mult': 0.833, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/20 22:32:07 INFO|
2022/09/20 22:32:26 INFO|{'vx_seed': 0, 'vir_mult': 2.38, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/20 22:32:27 INFO|
2022/09/20 22:32:47 INFO|{'vx_seed': 2, 'vir_mult': 0.833, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/20 22:32:48 INFO|
2022/09/20 22:33:08 INFO|{'vx_seed': 2, 'vir_mult': 2.38, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/20 22:33:10 INFO|Running reports
